# Automated Feature Engineering
#### See https://www.kaggle.com/willkoehrsen/automated-feature-engineering-tutorial

In [ ]:
# Required libraries
import pandas as pd
import numpy as np
import featuretools as ft
import warnings
warnings.filterwarnings('ignore')

In [23]:
# So we have the entities and 2 types of transaction
clients = pd.read_csv('./input/clients.csv',parse_dates=['joined'])
loans = pd.read_csv('./input/loans.csv',parse_dates=['loan_start','loan_end'])
payments = pd.read_csv('./input/payments.csv',parse_dates=['payment_date'])

In [24]:
# Create some features manually. First 2 new columns on the clients data
clients['join_month'] = clients['joined'].dt.month
clients['log_income'] = np.log(clients['income'])
clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


In [26]:
# calculate some stats about payments for each loan, then merge into loans
stats = payments.groupby('loan_id')['payment_amount'].agg(['sum'])
stats.columns = ['sum_payment_amount']

loans2 = loans.merge(stats, left_on='loan_id', right_index=True, how='left').head(10)
loans2.head()

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate,sum_payment_amount
0,46109,home,13672,0,10243,2002-04-16,2003-12-20,2.15,14008
1,46109,credit,9794,0,10984,2003-10-21,2005-07-17,1.25,7791
2,46109,home,12734,1,10990,2006-02-01,2007-07-05,0.68,12178
3,46109,cash,12518,1,10596,2010-12-08,2013-05-05,1.24,7681
4,46109,credit,14049,1,11415,2010-07-07,2012-05-21,3.13,16052


In [33]:
# Groupby clientid, calculate mean, max min previous loan size. Merge into clients
stats = loans2.groupby('client_id')['loan_amount','sum_payment_amount'].agg(['min','mean','max'])
stats.columns = ['min_loan_amount','mean_loan_amount','max_loan_amount','min_payment_amount','mean_payment_amount','max_payment_amount']
stats.head()
# Merge with clients
clients2 = clients.merge(stats, left_on='client_id', right_index=True, how='left').head(10)
clients2.head(20)

,client_id,joined,income,credit_score,join_month,log_income,min_loan_amount,mean_loan_amount,max_loan_amount,min_payment_amount,mean_payment_amount,max_payment_amount
0,46109,2002-04-16,172677,527,4,12.059178,4050.0,10364.7,14049.0,3422.0,9719.2,16052.0
1,49545,2007-11-14,104564,770,11,11.557555,NaN,NaN,NaN,NaN,NaN,NaN
2,41480,2013-03-11,122607,585,3,11.716739,NaN,NaN,NaN,NaN,NaN,NaN
3,46180,2001-11-06,43851,562,11,10.688553,NaN,NaN,NaN,NaN,NaN,NaN
4,25707,2006-10-06,211422,621,10,12.261611,NaN,NaN,NaN,NaN,NaN,NaN
5,39505,2011-10-14,153873,610,10,11.943883,NaN,NaN,NaN,NaN,NaN,NaN
6,32726,2006-05-01,235705,730,5,12.370336,NaN,NaN,NaN,NaN,NaN,NaN
7,35089,2010-03-01,131176,771,3,11.784295,NaN,NaN,NaN,NaN,NaN,NaN
8,35214,2003-08-08,95849,696,8,11.470529,NaN,NaN,NaN,NaN,NaN,NaN
9,48177,2008-06-09,190632,769,6,12.158100,NaN,NaN,NaN,NaN,NaN,NaN
